In [1]:
%load_ext autoreload
%aimport -torch, pandas, hydra, numpy, pickle, os
# %aimport src, src.metrics
%autoreload 2
import os
from hydra import initialize, compose
import pathlib
import pandas as pd
import numpy as np
import pickle
from tqdm.auto import tqdm

# from datasets.jobdataset import generate_dataset, _hfd5_from_dataframe
from src.data_process.job_hdf5 import hdf5_from_dataframe, get_career
import src.data_process.neg_sample as ng_sample
from src.utils.constants import DEFAULT_USER_COL,DEFAULT_ITEM_COL,DEFAULT_RATING_COL, DEFAULT_PREDICTION_COL
# from implicit_eval import microsoft_eval,model_infer_df
# from implicit.als import AlternatingLeastSquares
# from implicit.bpr import BayesianPersonalizedRanking
# from implicit.lmf import LogisticMatrixFactorization
from src.implicit_build import bpr, bpr, lmf
from src.metrics import ranking
from src.metrics.evaluate_ignite import model_infer2

with initialize(version_base=None, config_path="../../src/conf"):
    cfg = compose(config_name="config", overrides=['path.root=../../data/jobs'])

/Users/hao/.pyenv/versions/3.6.15/envs/torch-cpu5/lib/python3.6/site-packages/tqdm/auto.py:22: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [2]:
%aimport

Modules to reload:
all-except-skipped

Modules to skip:
torch, pandas, hydra, numpy, pickle, os


# Create the the data pipe

In [2]:
df_train_pos  = ng_sample.read_feather(pathlib.Path(cfg.path.root, cfg.file.train_pos))

In [3]:
df_train_pos[DEFAULT_RATING_COL] = 1

In [4]:
# df_train_pos[DEFAULT_USER_COL] = df_train_pos[DEFAULT_USER_COL].astype(str)
# df_train_pos[DEFAULT_ITEM_COL] = df_train_pos[DEFAULT_ITEM_COL].astype(str)
df_train_pos[DEFAULT_USER_COL] = df_train_pos[DEFAULT_USER_COL].astype("category")
df_train_pos[DEFAULT_ITEM_COL] = df_train_pos[DEFAULT_ITEM_COL].astype("category")

In [11]:
# hdf5_from_dataframe(df_train_pos, pathlib.Path(cfg.path.root, cfg.file.hdf5))

## Read data & Train

In [6]:
jobsid, usersid, user_job_app = get_career(pathlib.Path(cfg.path.root, cfg.file.hdf5))
model_path = "./models"

In [7]:
# user_job_app

In [8]:
# user_job_app = job_user_app.T.tocsr()
# bpr(model_path, user_job_app)
# als(model_path, user_job_app)
lmf(model_path, user_job_app)

100%|██████████| 30/30 [00:17<00:00,  1.67it/s]


In [6]:
# df_train_pos.userid.unique()

In [11]:
def read_train_gd_csv(data_testgd_path, usecols):
    test_gddf = pd.read_csv(data_testgd_path, usecols=usecols)
#     test_gddf[DEFAULT_USER_COL] = test_gddf[DEFAULT_USER_COL].astype('str')
#     test_gddf[DEFAULT_ITEM_COL] = test_gddf[DEFAULT_ITEM_COL].astype('str')
    return test_gddf

In [12]:
# pathlib.Path(cfg.path.root, cfg.file.train_neg)

In [13]:
df_test_ori = pd.read_feather(pathlib.Path(cfg.path.root, cfg.file.test))
# pathlib.Path(cfg.path.root, cfg.file.train_neg)
df_train =read_train_gd_csv('../../data/jobs/leave_one_train_neg.csv', usecols=[DEFAULT_USER_COL, DEFAULT_ITEM_COL, DEFAULT_RATING_COL])
# pathlib.Path(cfg.path.root, cfg.file.train_neg)
df_test =read_train_gd_csv('../../data/jobs/leave_one_test.csv', usecols=[DEFAULT_USER_COL, DEFAULT_ITEM_COL, DEFAULT_RATING_COL])


In [36]:
with open('./models/model_als.sav','rb') as pickle_in:
    model_als = pickle.load(pickle_in)
#     microsoft_eval(model, user_job_app, test_gddf=df_test, usersid=usersid, jobsid=jobsid, k=10, logger=None)
#     model.recommend(test_user_indexes, df_train[test_user_indexes], N=10)

In [9]:
with open('./models/model_bpr.sav','rb') as pickle_in:
    model_bpr = pickle.load(pickle_in)
with open('./models/model_lmf.sav','rb') as pickle_in:
    model_lmf = pickle.load(pickle_in)

In [12]:
# df_test_ori[DEFAULT_USER_COL] = df_test_ori[DEFAULT_USER_COL].astype('str')
# df_test_ori[DEFAULT_ITEM_COL] = df_test_ori[DEFAULT_ITEM_COL].astype('str')

In [13]:
apps_true = df_test_ori[df_test_ori['userid'].isin([1472090])]
# apps_true = df_test_ori

In [14]:
gt_pos_bpr, reco_ind_bpr, scores_bpr = model_infer2(df_true=apps_true, jobsid=jobsid, usersid=usersid, 
                                 model=model_bpr, u_i_matrix=user_job_app, n=cfg.params.neg_test+1)

In [15]:
gt_pos_als, reco_ind_als, scores_als = model_infer2(df_true=apps_true, jobsid=jobsid, usersid=usersid, 
                                 model=model_als, u_i_matrix=user_job_app, n=cfg.params.neg_test+1)
auc_test = ranking.AUC()
auc_test.compute(pd_scores=scores_als, gt_pos=gt_pos_als)

1.0

In [16]:
auc_test = ranking.AUC_k(10)
auc_test.compute(pd_scores=scores_als, gt_pos=gt_pos_als, pd_rank=reco_ind_als)

0.8888888888888888

In [80]:
def compute(pd_scores, gt_pos, gt_neg=None, **kwargs):
    
    if gt_neg is None:
        gt_neg = np.logical_not(gt_pos)
#     print(gt_pos.astype(np.bool))
    pos_scores = pd_scores[gt_pos.astype(np.bool)]
    print(pos_scores)
    neg_scores = pd_scores[gt_neg.astype(np.bool)]
    print(neg_scores)
    ui_scores = np.repeat(pos_scores, len(neg_scores))
    print(ui_scores)
    uj_scores = np.tile(neg_scores, len(pos_scores))
    print(uj_scores)
    if len(uj_scores)==0:
        return 0
    else:
        return (ui_scores > uj_scores).sum() / len(uj_scores)

In [89]:
pd_scores= np.asarray([.1, .2, .3, .4, .25])
gt_pos = np.asarray([0,1, 1, 0, 0])
# pd_scores = pd_scores[np.array([0,1])]

In [90]:
compute(pd_scores, gt_pos)

[0.2 0.3]
[0.1  0.4  0.25]
[0.2 0.2 0.2 0.3 0.3 0.3]
[0.1  0.4  0.25 0.1  0.4  0.25]


0.5

In [84]:
np.repeat(pd_scores, 2)

array([0.1, 0.1, 0.2, 0.2, 0.3, 0.3])

In [85]:
np.tile(pd_scores, 2)

array([0.1, 0.2, 0.3, 0.1, 0.2, 0.3])

In [19]:
def cal_metrics(apps_true, jobsid, usersid,model, u_i_matrix, n, metric:dict):
    gt_item, reco_ind, pd_scores = model_infer2(df_true=apps_true, jobsid=jobsid, usersid=usersid, 
                                 model=model, u_i_matrix=user_job_app, n=cfg.params.neg_test+1) 
    precision = metric['precision'].compute(gt_pos=gt_item, pd_rank=reco_ind)
    recall = metric['recall'].compute(gt_pos=gt_item, pd_rank=reco_ind)
    ndcg = metric['ndcg'].compute(gt_pos=gt_item, pd_rank=reco_ind)
    auc_k = metric['auc_k'].compute(gt_pos=gt_item, pd_scores=pd_scores, pd_rank=reco_ind)
    auc = metric['auc'].compute(gt_pos=gt_item, pd_scores=pd_scores)
    hit = metric['hit'].compute(gt_pos=gt_item)
#     return {'precision':precision, 'recall':recall, 'auc':auc, 'ndcg':ndcg}
#     return precision, recall
    return [precision, recall, ndcg, auc_k, auc, hit]

In [15]:
# aa= df_test_ori[df_test_ori['userid'].isin(['7','999368'])]
#  '1327949',
#  '543701',
#  '860552',
#  '298359' ])]
tqdm.pandas()


In [ ]:
def get_results(metric_value):
    precision_top = np.mean(np.array(list(metric_value.flat))[:,0])
    recall_top = np.mean(np.array(list(metric_value.flat))[:,1])
    ndcg_top = np.mean(np.array(list(metric_value.flat))[:,2])
    auc_top_k = np.mean(np.array(list(metric_value.flat))[:,3])
    auc = np.mean(np.array(list(metric_value.flat))[:,4])
    hit = np.mean(np.array(list(metric_value.flat))[:,5])
    print(f'Precision: {precision_top:.4f} \nRecall: {recall_top:.4f} \nNDCG: {ndcg_top:.4f} \nAUC_K: {auc_top_k:.4f} \
    \nAUC: {auc:.4f} \nHitRate: {hit:.4f}')

In [16]:
k=10
metric={'precision':ranking.Precision(k=k),'recall':ranking.Recall(k=k), 'ndcg':ranking.NDCG(k=k), \
        'auc_k':ranking.AUC_k(k=k),'auc':ranking.AUC(), 'hit':ranking.HitRate(k=k)}

In [33]:
metric_result = df_test_ori.groupby('userid').progress_apply(cal_metrics, jobsid=jobsid, usersid=usersid, 
                            model=model_bpr, u_i_matrix=user_job_app, 
                           n=cfg.params.neg_test+1, metric=metric)
metric_value_b = metric_result.values

100%|██████████| 74091/74091 [09:15<00:00, 133.27it/s]


In [34]:
get_results(metric_value_b)

Precision: 0.0141 
Recall: 0.1406 
NDCG: 0.0619 
AUC_K: 0.2875     
AUC: 0.7659 
HitRate: 0.5737


In [ ]:
metric_result = df_test_ori.groupby('userid').progress_apply(cal_metrics, jobsid=jobsid, usersid=usersid, 
                            model=model_als, u_i_matrix=user_job_app, 
                           n=cfg.params.neg_test+1, metric=metric)
metric_value_a = metric_result.values

 60%|█████▉    | 44144/74091 [05:27<04:12, 118.55it/s]

In [ ]:
get_results(metric_value_a)

In [20]:
metric_result = df_test_ori.groupby('userid').progress_apply(cal_metrics, jobsid=jobsid, usersid=usersid, 
                            model=model_lmf, u_i_matrix=user_job_app, 
                           n=cfg.params.neg_test+1, metric=metric)
metric_value_l = metric_result.values

100%|██████████| 74091/74091 [09:11<00:00, 134.31it/s]


In [31]:
get_results(metric_value_l)

Precision: 0.0121 
Recall: 0.1214 
NDCG: 0.0566 
AUC_K: 0.4099     
AUC: 0.8925 
HitRate: 0.7861
